## Siamene

### helper and processing

In [ ]:
!git clone https://github.com/H-TayyarMadabushi/AStitchInLanguageModels.git

In [ ]:
!git clone https://github.com/huggingface/transformers.git
%cd transformers/
!pip install --editable .
%cd /content/ 

Cloning into 'transformers'...
remote: Enumerating objects: 93527, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 93527 (delta 57), reused 77 (delta 43), pack-reused 93430
Receiving objects: 100% (93527/93527), 85.92 MiB | 17.32 MiB/s, done.
Resolving deltas: 100% (68576/68576), done.
/content/transformers
Obtaining file:///content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 6.6 MB 5.5 MB/s 
     |████████████████████████████████| 77 kB 6.2 MB/s 
     |████████████████████████████████| 596 kB 50.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Running setup.py develop for transformers
/content


In [ ]:
!pip install datasets
!pip install transformers
!pip install sentence_transformers

     |████████████████████████████████| 325 kB 5.2 MB/s 
     |████████████████████████████████| 136 kB 50.7 MB/s 
     |████████████████████████████████| 212 kB 49.1 MB/s 
     |████████████████████████████████| 1.1 MB 38.5 MB/s 
     |████████████████████████████████| 127 kB 34.4 MB/s 
     |████████████████████████████████| 271 kB 46.2 MB/s 
     |████████████████████████████████| 144 kB 56.6 MB/s 
     |████████████████████████████████| 94 kB 3.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 4.0 MB 3.9 MB/s 
     |████████████████████████████████| 880 kB 60.0 MB/s

In [ ]:
!git clone https://github.com/H-TayyarMadabushi/SemEval_2022_Task2-idiomaticity.git

Cloning into 'SemEval_2022_Task2-idiomaticity'...
remote: Enumerating objects: 123, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 123 (delta 48), reused 61 (delta 15), pack-reused 0
Receiving objects: 100% (123/123), 2.50 MiB | 9.29 MiB/s, done.
Resolving deltas: 100% (48/48), done.


In [ ]:
def load_csv( path ) : 
  header = None
  data   = list()
  with open( path, encoding='utf-8') as csvfile:
    reader = csv.reader( csvfile ) 
    for row in reader : 
      if header is None : 
        header = row
        continue
      data.append( row ) 
  return header, data
def tokenise_idiom( phrase ) :
  return 'ID' + re.sub( r'[\s|-]', '', phrase ).lower() + 'ID'

In [ ]:
import re
import os
import sys
import csv
import gzip
import math
import torch
import random
import numpy as np

from datetime                         import datetime
from torch.utils.data                 import DataLoader
from sklearn.metrics.pairwise         import paired_cosine_distances

from datasets                         import load_dataset
from transformers                     import AutoModelForMaskedLM
from transformers                     import AutoTokenizer
from sentence_transformers            import SentenceTransformer,  LoggingHandler, losses, models, util
from sentence_transformers.readers    import InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

sys.path.append( '/content/SemEval_2022_Task2-idiomaticity/SubTaskB/' )
from SubTask2Evaluator                import evaluate_submission

from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
import numpy as np
from keras import backend as K
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Activation, Convolution1D, GlobalAveragePooling1D, Input, Bidirectional, Concatenate, Lambda
import random
import sklearn.metrics

In [ ]:
train_samples = []
dev_samples   = []
test_samples  = []
for split in [ 'train', 'validation', 'test' ] :
      dataset = load_dataset( 'assin2', split=split, ignore_verifications=True )
      for elem in dataset :
        ## {'entailment_judgment': 1, 'hypothesis': 'Uma criança está segurando uma pistola de água', 'premise': 'Uma criança risonha está segurando uma pistola de água e sendo espirrada com água', 'relatedness_score': 4.5, 'sentence_pair_id': 1}
          score = float( elem['relatedness_score'] ) / 5.0 # Normalize score to range 0 ... 1
          inp_example = InputExample(texts=[elem['hypothesis'], elem['premise']], label=score)
          if split == 'validation':
            dev_samples.append(inp_example)
          elif split == 'test':
            test_samples.append(inp_example)
          elif split == 'train' :
            train_samples.append(inp_example)
          else :
              raise Exception( "Unknown split. Should be one of ['train', 'test', 'validation']." )

Using custom data configuration default


Generating train split:   0%|          | 0/6500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2448 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Dataset assin2 downloaded and prepared to /root/.cache/huggingface/datasets/assin2/default/1.0.0/8467f7acbda82f62ab960ca869dc1e96350e0e103a1ef7eaa43bbee530b80061. Subsequent calls will reuse this data.


Using custom data configuration default
Reusing dataset assin2 (/root/.cache/huggingface/datasets/assin2/default/1.0.0/8467f7acbda82f62ab960ca869dc1e96350e0e103a1ef7eaa43bbee530b80061)
Using custom data configuration default
Reusing dataset assin2 (/root/.cache/huggingface/datasets/assin2/default/1.0.0/8467f7acbda82f62ab960ca869dc1e96350e0e103a1ef7eaa43bbee530b80061)


In [ ]:
data_location = '/content/SemEval_2022_Task2-idiomaticity/SubTaskB/EvaluationData/'
outpath = '/content/models/'
dev_location                = os.path.join( data_location, 'dev.csv'                     ) 
eval_location               = os.path.join( data_location, 'eval.csv'                    ) 
dev_formated_file_location  = os.path.join( data_location, 'dev.submission_format.csv'   ) 
eval_formated_file_location = os.path.join( data_location, 'eval.submission_format.csv'   ) 
## Save tmp model here.  
outdir = os.path.join( outpath, 'pBERT' + '-' + str( 4 ) )

In [ ]:
!mkdir models

### pre_train model

In [ ]:
train_data=[]
for i in train_samples:
  train_data.append([i.texts[0],i.texts[1],i.label])

In [ ]:
tok=Tokenizer(split=' ',oov_token='UNK',char_level=False)

In [ ]:
x1=np.array([i[0] for i in train_data])
x2=np.array([i[1] for i in train_data])

In [ ]:
tok.fit_on_texts(x1)
tok.fit_on_texts(x2)

In [ ]:
x1=pad_sequences(tok.texts_to_sequences(x1), maxlen =32)

In [ ]:
x2=pad_sequences(tok.texts_to_sequences(x2), maxlen =32)

In [ ]:
y=np.array([i[2] for i in train_data])

In [ ]:
maxl=32
embed_size=128
inp1 = Input(shape=(maxl,))
inp2 = Input(shape=(maxl,))
embedding_layer =  Embedding(len(tok.word_index)+1, output_dim=embed_size, input_length=maxl,trainable=True, mask_zero=True)
emb1 = embedding_layer(inp1)
emb2 = embedding_layer(inp2)
lstm =  Bidirectional(LSTM(units=32, return_sequences=False))
rep1 = lstm(emb1)
rep2 = lstm(emb2)
l1_norm = lambda x: 1 - K.abs(x[0] - x[1])
merged = Lambda(function=l1_norm, output_shape=lambda x: x[0], name='L1_distance')([rep1, rep2])
simi = Dense(1, activation='sigmoid', name='similarity')(merged)
sia_model=Model([inp1, inp2], simi)

In [ ]:
sia_model.compile(loss = 'mean_squared_error', optimizer = "adam", metrics=["mse"])
sia_model.fit([x1, x2], y, epochs = 5, shuffle=True, batch_size = 16,)
# sia_model.save('models/')


Epoch 1/5
407/407 [==============================] - 50s 81ms/step - loss: 0.0272 - mse: 0.0272
Epoch 2/5
407/407 [==============================] - 31s 76ms/step - loss: 0.0109 - mse: 0.0109
Epoch 3/5
407/407 [==============================] - 31s 76ms/step - loss: 0.0066 - mse: 0.0066
Epoch 4/5
407/407 [==============================] - 31s 76ms/step - loss: 0.0047 - mse: 0.0047
Epoch 5/5
407/407 [==============================] - 31s 75ms/step - loss: 0.0036 - mse: 0.0036


In [ ]:
languages=['PT']

In [ ]:
def prepare_eval_data( location, languages, test_print=False ) :
  header, data = load_csv( location )
  sentence1s = list()
  sentence2s = list()
  for elem in data : 
    if not languages is None and not elem[ header.index( 'Language' ) ] in languages : 
      continue
    sentence1 = elem[ header.index( 'sentence1' ) ] 
    sentence2 = elem[ header.index( 'sentence2' ) ] 
    mwe1      = elem[ header.index( 'MWE1'      ) ] 
    mwe2      = elem[ header.index( 'MWE2'      ) ] 

    if test_print : 
      print( sentence1 ) 
      print( sentence2 ) 
      print( mwe1 ) 
      print( mwe2 ) 

    if mwe1 != 'None' : 
      replaced = re.sub( mwe1, tokenise_idiom( mwe1 ), sentence1, flags=re.I)
      assert replaced != sentence1
      sentence1 = replaced
    if mwe2 != 'None' : 
      replaced = re.sub( mwe1, tokenise_idiom( mwe2 ), sentence2, flags=re.I)
      assert replaced != sentence2
      sentence2 = replaced

    if test_print : 
      print( sentence1 ) 
      print( sentence2 ) 
      break

    sentence1s.append( sentence1 ) 
    sentence2s.append( sentence2 ) 

  return sentence1s, sentence2s

In [ ]:
sentences1, sentences2 = prepare_eval_data( dev_location, languages )

In [ ]:
s1=pad_sequences(tok.texts_to_sequences(np.array(sentences1)), maxlen =32)
s2=pad_sequences(tok.texts_to_sequences(np.array(sentences2)), maxlen =32)

In [ ]:
predictions=sia_model.predict([s1, s2],batch_size=16)

In [ ]:
def insert_to_submission( languages, settings, sims, location ) : 
  header, data = load_csv( location ) 
  sims = list( reversed( sims ) )
  ## Validate with length
  updatable = [ i for i in data if i[ header.index( 'Language' ) ] in languages and i[ header.index( 'Setting' ) ] in settings ]
  assert len( updatable ) == len( sims ) 

  ## Will update in sequence - if data is not in sequence must update one language / setting at a time. 
  started_update = False
  for elem in data : 
    if elem[ header.index( 'Language' ) ] in languages and elem[ header.index( 'Setting' ) ] in settings : 
      sim_to_insert = sims.pop()
      elem[-1] = sim_to_insert
      started_update = True
    else :  
      assert not started_update ## Once we start, we must complete. 
    if len( sims ) == 0 : 
      break 
  assert len( sims ) == 0 ## Should be done here. 

  return [ header ] + data ## Submission file must retain header. 

In [ ]:
def write_csv( data, location ) : 
  with open( location, 'w', encoding='utf-8') as csvfile:
    writer = csv.writer( csvfile ) 
    writer.writerows( data ) 
  print( "Wrote {}".format( location ) ) 
  return


In [ ]:
predictions=[i[0] for i in predictions]

In [ ]:
## Create submission file on the development set. 
submission_data = insert_to_submission( languages, [ 'pre_train' ], predictions, dev_formated_file_location )  
results_file    = os.path.join( outpath, 'dev.pre_train_results-' + '.csv' )
write_csv( submission_data, results_file )

## Evaluate development set. 
results = evaluate_submission( results_file, os.path.join( data_location, 'dev.gold.csv' ) )

Wrote /content/models/dev.pre_train_results-.csv


In [ ]:
results

[['Settings',
  'Languages',
  'Spearman Rank ALL',
  'Spearman Rank Idiom Data',
  'Spearman Rank STS Data'],
 ['pre_train', 'EN', None, None, None],
 ['pre_train',
  'PT',
  0.6875906001920474,
  0.21433564589440202,
  0.8811952550883991],
 ['pre_train', 'EN,PT', None, None, None],
 ['fine_tune', 'EN', None, None, None],
 ['fine_tune', 'PT', None, None, None],
 ['fine_tune', 'EN,PT', None, None, None]]

### Fine Tune

In [ ]:
def _parse_train_data( train_data_location, languages, tokenize=True ) :

  header, train_data = load_csv( train_data_location )
  
  train_data_with_labels                 = list()
  train_data_requiring_labels            = list()
  need_predictions_for_train_data_labels = list()

  # ['ID', 'MWE1', 'MWE2', 'Language', 'sentence_1', 'sentence_2', 'sim', 'alternative_1', 'alternative_2']

  skipped = 0 
  for elem in train_data :

    if not elem[ header.index( 'Language' ) ] in languages :
      skipped += 1
      continue

    mwe1          = elem[ header.index( 'MWE1'          ) ] 
    mwe2          = elem[ header.index( 'MWE2'          ) ] 
    
    this_sim      = elem[ header.index( 'sim'           ) ]
    sentence_1    = elem[ header.index( 'sentence_1'    ) ]
    sentence_2    = elem[ header.index( 'sentence_2'    ) ]
    alternative_1 = elem[ header.index( 'alternative_1' ) ]
    alternative_2 = elem[ header.index( 'alternative_2' ) ]

    ## Remove below if you do not want to tokenize with idiom tokens!
    if tokenize : 
      if mwe1 != 'None' : 
        replaced = re.sub( mwe1, tokenise_idiom( mwe1 ), sentence_1, flags=re.I)
        assert replaced != sentence_1
        sentence_1 = replaced
      if mwe2 != 'None' : 
        replaced = re.sub( mwe1, tokenise_idiom( mwe2 ), sentence_2, flags=re.I)
        assert replaced != sentence_2
        sentence_2 = replaced
  
   
    if this_sim != 'None' :
      tmp = float( this_sim ) 
      train_data_with_labels.append( [ sentence_1, sentence_2, this_sim ] ) 
      continue
    train_data_requiring_labels.append( [ sentence_1, sentence_2 ] ) 
    need_predictions_for_train_data_labels.append( [ alternative_1, alternative_2 ] )

  assert len( need_predictions_for_train_data_labels ) == len( train_data_requiring_labels )
  assert len( train_data ) == len( need_predictions_for_train_data_labels ) + len( train_data_with_labels ) + skipped

  return train_data_with_labels, train_data_requiring_labels, need_predictions_for_train_data_labels 


In [ ]:
def _get_predictions_for_train_data_labels( model, data ) :
  simi_scores=model.predict(data)
  
  return simi_scores

In [ ]:
def generate_train_data( train_data_location, languages ) :
  
  train_data_with_labels, train_data_requiring_labels, need_predictions_for_train_data_labels = _parse_train_data( train_data_location, languages )
  x1=np.array([i[0] for i in need_predictions_for_train_data_labels])
  x2=np.array([i[1] for i in need_predictions_for_train_data_labels])
  x1=pad_sequences(tok.texts_to_sequences(np.array(x1)), maxlen =32)
  x2=pad_sequences(tok.texts_to_sequences(np.array(x2)), maxlen =32)
  sims = _get_predictions_for_train_data_labels( sia_model, [x1,x2] )

  train_data_requiring_labels_with_labels = list()
  for index in range( len( train_data_requiring_labels ) ) : 
    train_data_requiring_labels_with_labels.append( [ train_data_requiring_labels[index][0], train_data_requiring_labels[index][1], sims[index][0] ] )

  train_data =train_data_with_labels + train_data_requiring_labels_with_labels
  assert all( [ (len(i) == 3) for i in train_data ] )
  return train_data

In [ ]:
train_data_location = 'SemEval_2022_Task2-idiomaticity/SubTaskB/TrainData/train_data.csv'
out_location        = 'models/FineTune/'


In [ ]:
train_datas = generate_train_data( train_data_location,languages )

In [ ]:
train_data=train_data+train_datas
len(train_data)

8347

In [ ]:
x1=np.array([i[0] for i in train_data])
x2=np.array([i[1] for i in train_data])


In [ ]:
tok.fit_on_texts(x1)
tok.fit_on_texts(x2)
x1=pad_sequences(tok.texts_to_sequences(x1), maxlen =32)
x2=pad_sequences(tok.texts_to_sequences(x2), maxlen =32)
y=np.array([float(i[2]) for i in train_data])

In [ ]:
maxl=32
embed_size=128
inp1 = Input(shape=(maxl,))
inp2 = Input(shape=(maxl,))
embedding_layer =  Embedding(len(tok.word_index)+1, output_dim=embed_size, input_length=maxl, trainable=True, mask_zero=True)
emb1 = embedding_layer(inp1)
emb2 = embedding_layer(inp2)
lstm =  Bidirectional(LSTM(units=32, return_sequences=False))
rep1 = lstm(emb1)
rep2 = lstm(emb2)
l1_norm = lambda x: 1 - K.abs(x[0] - x[1])
merged = Lambda(function=l1_norm, output_shape=lambda x: x[0], name='L1_distance')([rep1, rep2])
simi = Dense(1, activation='sigmoid', name='similarity')(merged)
sia_model=Model([inp1, inp2], simi)

In [ ]:
sia_model.compile(loss = 'mean_squared_error', optimizer = "adam", metrics=["mse"])
sia_model.fit([x1, x2], y, epochs = 5, shuffle=True, batch_size = 16,)

Epoch 1/5
522/522 [==============================] - 59s 83ms/step - loss: 0.0265 - mse: 0.0265
Epoch 2/5
522/522 [==============================] - 44s 85ms/step - loss: 0.0098 - mse: 0.0098
Epoch 3/5
522/522 [==============================] - 43s 83ms/step - loss: 0.0060 - mse: 0.0060
Epoch 4/5
522/522 [==============================] - 43s 83ms/step - loss: 0.0045 - mse: 0.0045
Epoch 5/5
522/522 [==============================] - 44s 83ms/step - loss: 0.0035 - mse: 0.0035


In [ ]:
predictions=sia_model.predict([s1, s2],batch_size=16)

In [ ]:
predictions=[i[0] for i in predictions]

In [ ]:
submission_data = insert_to_submission( languages, [ 'fine_tune' ], predictions, dev_formated_file_location )  
results_file    = os.path.join( outpath, 'dev.combined_results-'+ '.csv' )
write_csv( submission_data, results_file )

Wrote /content/models/dev.combined_results-.csv


In [ ]:
results = evaluate_submission( results_file, os.path.join( data_location, 'dev.gold.csv' ) )

In [ ]:
results

[['Settings',
  'Languages',
  'Spearman Rank ALL',
  'Spearman Rank Idiom Data',
  'Spearman Rank STS Data'],
 ['pre_train', 'EN', None, None, None],
 ['pre_train', 'PT', None, None, None],
 ['pre_train', 'EN,PT', None, None, None],
 ['fine_tune', 'EN', None, None, None],
 ['fine_tune',
  'PT',
  0.5802636532244051,
  0.2421507602908165,
  0.3100324445905441],
 ['fine_tune', 'EN,PT', None, None, None]]